## Visual QC notebook

Execute this notebook from the command line using `papermill`. The first argument is the input notebook location, the second argument is the notebook output location, and the remaining arguments for indicators, models, and scenarios should be passed directly from the Prefect flow parameters. Note that each parameter is preceded by the `-r` tag to designate a raw string, and the multiple items are enclosed in quotes and separated by a single white space.


```papermill path/to/repo/indicators/visual_qc.ipynb path/to/qc/output/output.ipynb -r indicators "rx1day" -r models "GFDL-ESM4 CESM2 TaiESM1" -r "historical ssp126 ssp245 ssp370 ssp585"```



In [8]:
# this cell is tagged "parameters" and contains default values for this notebook
# these values are read when executing the notebook using papermill
# parameters injected from the command line that differ from these defaults will override the values in this cell
# use a raw string with values separated by blank space; this matches the raw string parameter input from the command line

indicators = "rx1day"
models = "GFDL-ESM4 CESM2 TaiESM1"
scenarios = "historical ssp126 ssp245 ssp370 ssp585"


In [9]:
indicators_, models_, scenarios_ = indicators.split(" "), models.split(" "), scenarios.split(" ")

print(indicators_)
print(models_)
print(scenarios_)

['rx1day']
['GFDL-ESM4', 'CESM2', 'TaiESM1']
['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
